# HyperDrive

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project", subscription_id='8bad5f66-e234-46b1-b07f-d3cfd6255f67')
exp = Experiment(workspace=ws, name="udacity-project-kenny")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: eastus
Subscription id: 8bad5f66-e234-46b1-b07f-d3cfd6255f67
Resource group: udacity-project


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-project"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("I found an exisiting cluster so I'm using it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

I found an exisiting cluster so I'm using it.


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
params = {
    "C": uniform(0.0, 1.0),
    "max_iter": choice(50, 100, 150, 200, 250, 300)
}
ps = RandomParameterSampling(params)

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=compute_target,
    entry_script="train.py"
)

primary_metric_goal = PrimaryMetricGoal("MAXIMIZE")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=primary_metric_goal,
    max_concurrent_runs=4,
    max_total_runs=40
)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
params = best_run.get_details()['runDefinition']['arguments']

print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy'])
print('C:', params[1])
print('max_iter:', params[3])

# save model
# model = best_run.register_model(model_name='model', model_path='outputs/model.pkl')
# print(f"Model {model.name}.v{model.version} correctly saved")
best_run.download_file("./outputs/model.joblib", "logreg_best_model.joblib")

Id of the best run:  HD_b81e5cbe-63ff-474d-999e-cc9e1f95bb75_12
Accuracy of the best run: 0.9167273610099539
C: 0.25137065561901684
max_iter: 200


# AutoML

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
5247,40,0,0,0,0,11,1,105,2,3,...,0,0,0,0,0,0,0,1,0,1
1819,26,0,0,1,0,10,2,1003,3,999,...,1,0,0,1,0,0,0,0,0,1
18533,30,1,0,1,1,5,5,114,2,999,...,1,0,0,0,1,0,0,0,0,0
17129,48,1,0,0,0,5,2,284,2,999,...,1,0,0,1,0,0,0,0,0,0
11061,28,1,0,1,0,6,1,273,5,999,...,1,0,1,0,0,0,0,0,0,0


In [21]:
exp2 = Experiment(workspace=ws, name="udacity-project-automl")

Running on local machine


ValidationException: ValidationException:
	Message: Please install specific versions of packages: We validated this package with zipp<=3.1.0 but your environment has zipp 3.2.0.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Please install specific versions of packages: We validated this package with zipp<=3.1.0 but your environment has zipp 3.2.0.",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
                "code": "IncompatibleOrMissingDependency"
            }
        },
        "reference_code": "65e7ad52-ad1f-4915-85ad-440d0a5e221f"
    }
}

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  l1_ratio=0.836734693877551,
                                                                                                  learning_rate='constant',
                                                                                                  loss='modified_huber',
                     

In [13]:
y_pred = fitted_automl_model.predict(x_test)

In [14]:
cc)

Test set accuracy: 0.9176984705025492


In [15]:
b")

['fitted_automl_model.joblib']

In [16]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

